 # Git 

In [ ]:
!git clone https://github.com/DmitryUlyanov/deep-image-prior
!mkdir deep_image_prior
!mv ./deep-image-prior/* ./deep_image_prior/
!rm -rf ./deep-image-prior

# Models & Libraries

In [ ]:
import numpy as np
import cv2
import torch
from torch import optim
from torch.autograd import Function
from torchvision import models
import os
from os import listdir
from os.path import isfile, join
import pickle 

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from utils import *
from deep_image_prior.utils.perceptual_loss.perceptual_loss import *
from deep_image_prior.utils.common_utils import *

%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import print_function
%matplotlib inline

import argparse
from deep_image_prior.models import *
import torch.optim
torch.backends.cudnn.enabled = True

dtype = torch.cuda.FloatTensor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Choose net type
import torchvision.models as models
# pretrained_net = 'vgg19_caffe' 
# assert pretrained_net in ['alexnet_caffe', 'vgg19_caffe', 'vgg16_caffe']
pretrained_net = models.resnet18(pretrained=True)
pretrained_net.eval()

In [ ]:
def get_resnet_preprocessor(imsize):
    preprocess = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    return preprocess

In [ ]:
# Target imsize 
imsize = 227 if pretrained_net == 'alexnet_caffe' else 224

# Something divisible by a power of two
imsize_net = 256

# VGG and Alexnet need input to be correctly normalized
preprocess = get_resnet_preprocessor(imsize)

cnn = pretrained_net.to(device)

# Setting Work Directories

In [ ]:
dir = '/media/heatmaps/null'

# Change before run.
data_dir = '/media/data/null/Data'

# Choose the setting for generating image samples.
image_constructors = {
    'repeated_patch_4': repeated_patch_image,
    'repeated_patch_2': repeated_two_patch,
    'single_patch': single_patch_image_constructor,
    'two_patch': image_constructor}

mode = 'two_patch'
assert(mode in image_constructors)

patches_files = [f for f in listdir(data_dir) if isfile(join(data_dir, f))]
data_list = []

count_ = 0
for file_name in patches_files:    
  file_dir = join(data_dir, file_name)
  f = open(file_dir, "rb")
  # [patch_positions, random_bg, map_idxs]  
  data_list += [pickle.load(f)]
  f.close()

images = []
for data in data_list:
  images += image_constructors[mode](data)

In [ ]:
def save_data_image(dir, name, hm, step):

  heatmap = hm.squeeze().cpu()
  if len(heatmap.shape) != 2:
    if heatmap.shape[0] == 3:
        heatmap = torch.mean(heatmap, 0).squeeze()
    else:
        heatmap = torch.mean(heatmap, 2).squegeze()
  
  if torch.max(abs(heatmap)) > 1:
      heatmap /= 255

  data_dir = join(dir, 'data')
  os.makedirs(data_dir, exist_ok=True)
  full_dir = join(data_dir, name)
  f = open(full_dir, 'wb')
  pickle.dump(heatmap, f)
  f.close()
    
  heatmap = abs(heatmap)
  heatmap = torch.clamp(heatmap*1.5, min=0, max=1)
    
  image_number = int(name.split('_')[-1].split('.')[1])
  if image_number < 50: 
    image_dir = join(dir, 'images')
    os.makedirs(image_dir, exist_ok=True)
    plt.imsave(join(image_dir, name + '.png'), heatmap.numpy(), cmap='Greys', format='png')
    plt.imsave(join(image_dir, name + '.pdf'), heatmap.numpy(), cmap='Greys', format='pdf')

def visualize_image(img):
  plt.imshow(img.cpu().permute(1, 2, 0).detach().numpy())

# GradCAM

In [ ]:
!pip install git+https://github.com/jacobgil/pytorch-grad-cam.git
!pip install ttach
!pip install grad-cam

In [ ]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
def attrib_GradCAM(target_idx):
    target_dir_name = 'target'
    if target_idx == 1:
        target_dir_name = 'second_target'
        
    target_dir = os.path.join(dir, 'GradCAM/', target_dir_name)
    os.makedirs(target_dir, exist_ok=True)
    
    imsize = 227 if pretrained_net == 'alexnet_caffe' else 224
    cnn = models.resnet18(pretrained=True).to(device).eval()
    counter = 0

    for input_tensor in tqdm(images):

      input_tensor.grad = None    
      cnn.zero_grad()
      grad_cam = GradCAM(model=cnn, target_layers=[cnn.layer4[-1]], use_cuda=True)
      input_tensor = torch.tensor(input_tensor).reshape(1, 3, imsize, imsize).cuda()
      input = preprocess(input_tensor)
      input.requires_grad = True

      target_index = int(data_list[counter][2][target_idx])

      mask = grad_cam(input, target_index)
      mask = mask[0, :]

      img_bgr = show_cam_on_image(input_tensor.squeeze().permute(1, 2, 0).cpu().numpy(), mask, use_rgb=True)
      hm_name = str(counter) + "_target:" + str(target_index) + "_" + patches_files[counter]

      save_data_image(target_dir, hm_name, torch.Tensor(mask), counter)
      counter += 1

In [ ]:
attrib_GradCAM(0)
attrib_GradCAM(1)

# GradCAMPlusPlus

In [ ]:
!pip install git+https://github.com/jacobgil/pytorch-grad-cam.git
!pip install ttach
!pip install grad-cam

In [ ]:
from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
def attrib_GradCAMPlusPlus(target_idx):
    target_dir_name = 'target'
    if target_idx == 1:
        target_dir_name = 'second_target'
    
    for l in range(4):
        target_dir = os.path.join(dir, 'GradCAMPlusPlus/', 'layer'+str(l+1), target_dir_name)
        os.makedirs(target_dir, exist_ok=True)

        imsize = 227 if pretrained_net == 'alexnet_caffe' else 224
        cnn = models.resnet18(pretrained=True).to(device).eval()
        counter = 0

        for input_tensor in tqdm(images):

          input_tensor.grad = None    
          cnn.zero_grad()
          grad_target_layers = [cnn.layer1[-1], cnn.layer2[-1], cnn.layer3[-1], cnn.layer4[-1]]            
          grad_cam = GradCAMPlusPlus(model=cnn, target_layers=[grad_target_layers[l]], use_cuda=True)
          input_tensor = torch.tensor(input_tensor).reshape(1, 3, imsize, imsize).cuda()
          input = preprocess(input_tensor)
          input.requires_grad = True

          target_index = int(data_list[counter][2][target_idx])

          mask = grad_cam(input, target_index)
          mask = mask[0, :]

          img_bgr = show_cam_on_image(input_tensor.squeeze().permute(1, 2, 0).cpu().numpy(), mask)

          hm_name = str(counter) + "_target:" + str(target_index) + "_" + patches_files[counter]

          save_data_image(target_dir, hm_name, torch.Tensor(mask), counter)
          counter += 1

In [ ]:
attrib_GradCAMPlusPlus(0)
attrib_GradCAMPlusPlus(1)

# FullGrad

In [ ]:
!git clone https://github.com/idiap/fullgrad-saliency.git
!mv fullgrad-saliency/* ./

In [ ]:
from saliency.fullgrad import FullGrad
from misc_functions import NormalizeInverse

In [ ]:
from misc_functions import *
def attrib_FullGrad(target_idx):
    
    target_dir_name = 'target'
    if target_idx == 1:
        target_dir_name = 'second_target'
        
    target_dir = os.path.join(dir, 'FullGrad/', target_dir_name)
    os.makedirs(target_dir, exist_ok=True)

    imsize = 227 if pretrained_net == 'alexnet_caffe' else 224
    
    cnn = models.resnet18(pretrained=True).to(device).eval()
    counter = 0

    unnormalize = NormalizeInverse(mean = [0.485, 0.456, 0.406],
                               std = [0.229, 0.224, 0.225])
    fullgrad = FullGrad(cnn)
    for input_tensor in tqdm(images):

      cnn.zero_grad()

      input_tensor = torch.tensor(input_tensor).permute(2, 0, 1).reshape(1, 3, imsize, imsize).cuda()
      input = preprocess(input_tensor)
      input.requires_grad = True

      target_index = int(data_list[counter][2][target_idx])
      target_index_tensor = torch.Tensor([[target_index]]).type(torch.int64).cuda()


      mask = fullgrad.saliency(input, target_index_tensor)
      mask = mask[0]

      hm_name = str(counter) + "_target:" + str(target_index) + "_" + patches_files[counter]
      save_data_image(target_dir, hm_name, torch.tensor(mask), counter)
      counter += 1

In [ ]:
attrib_FullGrad(0)
attrib_FullGrad(1)

# IBA

## Download ImageNET

In [ ]:
%mkdir /content/data
%cd /content/data
!wget -c https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar &
%mkdir validation && mv ILSVRC2012_img_val.tar validation/ && cd validation && tar -xvf ILSVRC2012_img_val.tar
%cd validation
!sh /content/drive/MyDrive/CAMP/ImageNet-Scripts/valprep.sh
%cd /content/

## Actual Method

In [ ]:
!pip install git+https://github.com/BioroboticsLab/IBA

In [ ]:
from IBA.pytorch import IBA, tensor_to_np_img, get_imagenet_folder, imagenet_transform
from IBA.utils import plot_saliency_map, to_unit_interval, load_monkeys

from torch.utils.data import DataLoader
from torchvision import models
import torch

def attrib_IBA(target_idx):
    
    target_dir_name = 'target'
    if target_idx == 1:
        target_dir_name = 'second_target'
        
    for l in range(3, 4):
        target_dir = os.path.join(dir, 'IBA/', 'layer'+str(l+1), target_dir_name)
        os.makedirs(target_dir, exist_ok=True)

        counter = 0

        imagenet_dir = '/media/validation'

        # Load model
        device = 'cuda:0' if  torch.cuda.is_available() else 'cpu'
        model = models.resnet18(pretrained=True).to(device).eval()

        imsize = 227 if model == 'alexnet_caffe' else 224
        
        # Add a Per-Sample Bottleneck at layer conv4_1
        grad_target_layers = [model.layer1[-1], model.layer2[-1], model.layer3[-1], model.layer4[-1]] 
        iba = IBA(grad_target_layers[l])

        # Estimate the mean and variance of the feature map at this layer.
        val_set = get_imagenet_folder(imagenet_dir)
        val_loader = DataLoader(val_set, batch_size=64, shuffle=True, num_workers=4)
        iba.estimate(model, val_loader, n_samples=5000, progbar=True)

        for input_tensor in tqdm(images):

          model.zero_grad()

          input_tensor = torch.tensor(input_tensor).reshape(1, 3, imsize, imsize).cuda()
          input = preprocess(input_tensor)
          input.requires_grad = True

          target_index = int(data_list[counter][2][target_idx])
          model_loss_closure = lambda x: -torch.log_softmax(model(x), 1)[:, target_index].mean()

          saliency_map = iba.analyze(input.to(device), model_loss_closure, beta=10)

          model_loss_closure = lambda x: -torch.log_softmax(model(x), 1)[:, target_index].mean()
          heatmap = iba.analyze(input.to(device), model_loss_closure )

          hm_name = str(counter) + "_target:" + str(target_index) + "_" + patches_files[counter]
          save_data_image(target_dir, hm_name, torch.tensor(heatmap), counter)
          counter += 1

In [ ]:
attrib_IBA(0)
attrib_IBA(1)

# GuidedBackProp, IntegratedGradients, DeepLiftShap


In [ ]:
!pip install captum

In [ ]:
from captum.attr import GuidedBackprop
from captum.attr import DeepLiftShap
from captum.attr import IntegratedGradients
from tqdm.notebook import tqdm
import cv2

In [ ]:
from resnet import *
def attrib_captum(target_idx, method):
    
    assert method in ['IntegratedGradients', 'GuidedBackProp', 'DeepLiftShap']
    
    target_dir_name = 'target'
    if target_idx == 1:
        target_dir_name = 'second_target'
    
    target_dir = os.path.join(dir, method, target_dir_name)
    os.makedirs(target_dir, exist_ok=True)
    imsize = 227 if pretrained_net == 'alexnet_caffe' else 224
    counter = 0
    
    
    for input_tensor in tqdm(images):

      if method == 'DeepLiftShap':
          #For DeepLiftShape, place resnet.py and replace cnn with the one below.
          cnn = resnet18(pretrained=True).to(device)
      else:    
          cnn = models.resnet18(pretrained=True).to(device).eval()

      cnn.zero_grad()

      input_tensor = torch.tensor(input_tensor).reshape(1, 3, imsize, imsize).cuda()
      input = preprocess(input_tensor)
      input.requires_grad = True

      target_index = int(data_list[counter][2][target_idx])

      if method == 'IntegratedGradients':        
          mask = IntegratedGradients(cnn).attribute(input, target=target_index)
      elif method == 'GuidedBackProp':                
          mask = GuidedBackprop(cnn).attribute(input, target=target_index)
      elif method == 'DeepLiftShap':                
          base_dist = torch.clamp(torch.zeros([10, 3, imsize, imsize]).normal_(mean=0.5, std=0.1).type(dtype), 0, 1).to(device)
          mask = DeepLiftShap(cnn).attribute(input, target=target_index, baselines=base_dist)

      mask = mask[0, :]
      mask = mask/torch.max(mask)

      hm_name = str(counter) + "_target:" + str(target_index) + "_" + patches_files[counter]
      save_data_image(target_dir, hm_name, mask.detach(), counter)
      counter += 1


In [ ]:
methods = ['IntegratedGradients', 'GuidedBackProp', 'DeepLiftShap']

for method in methods:
    for target in [0, 1]:
        print(method, 'target:', target)
        attrib_captum(target, method)

# GuidedBackProp, Gradient, ExtreamlPertubation

In [ ]:
!pip install torchray

In [ ]:
from torchray.attribution.guided_backprop import guided_backprop
from torchray.attribution.extremal_perturbation import extremal_perturbation
from torchray.attribution.grad_cam import grad_cam
from torchray.attribution.gradient import gradient

In [ ]:
def attrib_torchray(target_idx, method):
    
    assert method in ['ExtremalPertubation', 'Gradient']
    
    target_dir_name = 'target'
    if target_idx == 1:
        target_dir_name = 'second_target'
    
    target_dir = os.path.join(dir, method, target_dir_name)
    os.makedirs(target_dir, exist_ok=True)

    imsize = 227 if pretrained_net == 'alexnet_caffe' else 224
    cnn = models.resnet18(pretrained=True).to(device).eval()
    counter = 0

    for input_tensor in tqdm(images):
      cnn.zero_grad()

      input_tensor = torch.tensor(input_tensor).reshape(1, 3, imsize, imsize).cuda()
      input = preprocess(input_tensor)
      input.requires_grad = True

      target_index = int(data_list[counter][2][target_idx])

      if method == 'Gradient':
        mask = gradient(cnn, input, target_index)
      elif method == 'ExtremalPertubation':
        mask, _ = extremal_perturbation(cnn, input, target_index)
        
      mask = mask[0, :]
      mask = mask/torch.max(mask)

      hm_name = str(counter) + "_target:" + str(target_index) + "_" + patches_files[counter]
      save_data_image(target_dir, hm_name, mask, counter)
      counter += 1


In [ ]:
methods = ['ExtremalPertubation', 'Gradient']

for method in methods:
    for target in [0, 1]:
        print(method, 'target:', target)
        attrib_torchray(target, method)

# Visaulize Heatmaps

In [ ]:
from os import listdir
from os.path import isfile, join
import pickle 

data_dir = join(target_dir, 'data')

hm_files = [f for f in listdir(data_dir) if isfile(join(data_dir, f))]
data = []

for file_name in hm_files:
  file_dir = join(data_dir, file_name)
  f = open(file_dir, "rb")
  data += [pickle.load(f)]
  f.close()

In [ ]:
import matplotlib.pyplot as plt

def plot_heatmap(hm):
  
  heatmap = hm.squeeze().cpu()
  if len(heatmap.shape) != 2:
    heatmap = torch.sum(heatmap, 0).squeeze()
  
  plt.imshow(heatmap.numpy(), cmap='Reds') 


In [ ]:
plot_heatmap(data[0])

# Score

In [ ]:
print('Method: Class Sensitivity, Null Player')


method_dir = '/media/heatmaps/double'
methods = ['DeepLiftShap', 'ExtremalPertubation', 'FullGrad', 'GradCAM', 'Gradient',
          'GuidedBackProp', 'IntegratedGradients', 'GradCAMPlusPlus/layer1',
          'GradCAMPlusPlus/layer2', 'GradCAMPlusPlus/layer3', 'GradCAMPlusPlus/layer4', 'IBA/layer1',
          'IBA/layer2', 'IBA/layer3', 'IBA/layer4']


for m in methods:
    first_target_dir = join(method_dir, m, 'target/data')
    second_target_dir = join(method_dir, m, 'second_target/data')
    
    #################### READ HEATMAPS ####################
    first_hm_files = sorted([f for f in listdir(first_target_dir) if isfile(join(first_target_dir, f))])
    first_hm_list = []

    for file_name in sorted(first_hm_files):
      file_dir = join(first_target_dir, file_name)
      f = open(file_dir, "rb")
      first_hm_list += [pickle.load(f)]
      f.close()
        
    second_hm_files = sorted([f for f in listdir(second_target_dir) if isfile(join(second_target_dir, f))])
    second_hm_list = []

    for file_name in sorted(second_hm_files):
      file_dir = join(second_target_dir, file_name)
      f = open(file_dir, "rb")
      second_hm_list += [pickle.load(f)]
      f.close()    


    #################### CALCULATE SCORES ####################
    class_sensitivity = []
    null_player = []

    counter = 0
    for first_hm, second_hm in zip(first_hm_list, second_hm_list):
        index = second_hm_files[counter].split('_')[2]
        temp_data = data_list[patches_files.index(index)]

        null_player += [get_null_player(first_hm, temp_data).item()]
        class_sensitivity += [get_class_sensitivity([first_hm, second_hm], temp_data).item()]

        counter += 1

    print(m + ': ', np.array(class_sensitivity).mean(), ', ', np.array(null_player).mean())

In [ ]:
print('Method: Null Player')


method_dir = '/media/heatmaps/null'
methods = ['DeepLiftShap', 'ExtremalPertubation', 'FullGrad', 'GradCAM', 'Gradient',
          'GuidedBackProp', 'IntegratedGradients', 'GradCAMPlusPlus/layer1',
          'GradCAMPlusPlus/layer2', 'GradCAMPlusPlus/layer3', 'GradCAMPlusPlus/layer4', 'IBA/layer1',
          'IBA/layer2', 'IBA/layer3', 'IBA/layer4']


for m in methods:
    first_target_dir = join(method_dir, m, 'second_target/data')
    
    #################### READ HEATMAPS ####################
    first_hm_files = sorted([f for f in listdir(first_target_dir) if isfile(join(first_target_dir, f))])
    first_hm_list = []

    for file_name in sorted(first_hm_files):
      file_dir = join(first_target_dir, file_name)
      f = open(file_dir, "rb")
      first_hm_list += [pickle.load(f)]
      f.close()   

    #################### CALCULATE SCORES ####################
    null_player = []

    counter = 0
    for first_hm in zip(first_hm_list):
        index = first_hm_files[counter].split('_')[2]
        temp_data = data_list[patches_files.index(index)]

        null_player += [1/get_null_player(first_hm[0], temp_data).item()]

        counter += 1

    print(m + ': ', np.array(null_player).mean())

In [ ]:
from torch.nn.functional import softmax
from scipy.stats import spearmanr

print("Method: single patch")

method_dir = '/media/heatmaps/single/'
methods = ['DeepLiftShap', 'ExtremalPertubation', 'FullGrad', 'GradCAM', 'Gradient',
          'GuidedBackProp', 'IntegratedGradients', 'GradCAMPlusPlus/layer1',
          'GradCAMPlusPlus/layer2', 'GradCAMPlusPlus/layer3', 'GradCAMPlusPlus/layer4', 'IBA/layer1',
          'IBA/layer2', 'IBA/layer3', 'IBA/layer4']


for m in methods:
    first_target_dir = join(method_dir, m, 'target/data')
    second_target_dir = join(method_dir, m, 'second_target/data')

    #################### READ HEATMAPS ####################
    first_hm_files = sorted([f for f in listdir(first_target_dir) if isfile(join(first_target_dir, f))])
    first_hm_list = []

    for file_name in sorted(first_hm_files):
      file_dir = join(first_target_dir, file_name)
      f = open(file_dir, "rb")
      first_hm_list += [pickle.load(f)]
      f.close()


    second_hm_files = sorted([f for f in listdir(second_target_dir) if isfile(join(second_target_dir, f))])
    second_hm_list = []

    for file_name in sorted(second_hm_files):
      file_dir = join(second_target_dir, file_name)
      f = open(file_dir, "rb")
      second_hm_list += [pickle.load(f)]
      f.close()    


    #################### CALCULATE SCORES ####################
    first_target_scores = []
    second_target_scores = []
    first_target_random = []
    second_target_random = []


    # counter = 0
    for i in range(len(first_hm_list)):

        index = second_hm_files[i].split('_')[2]
        temp_data = data_list[patches_files.index(index)]

        t1 = single_patch_hm_sum(abs(first_hm_list[i]), temp_data)
        t2 = single_patch_hm_sum(abs(second_hm_list[i]), temp_data)
        
        first_target_scores.append(t1[0])
        second_target_scores.append(t2[0])

    corr, pval = spearmanr(first_target_scores, second_target_scores)
    print(m, ': ', corr)


In [ ]:
from torch.nn.functional import softmax
from scipy.stats import spearmanr

print("Method: Repeated")

method_dir = '/media/heatmaps/repeated-new/'
methods = ['DeepLiftShap', 'ExtremalPertubation', 'FullGrad', 'GradCAM', 'Gradient',
          'GuidedBackProp', 'IntegratedGradients', 'GradCAMPlusPlus/layer4', 'IBA/layer4']

for m in methods:
    first_target_dir = join(method_dir, m, 'second_target/data')

    #################### READ HEATMAPS ####################
    first_hm_files = sorted([f for f in listdir(first_target_dir) if isfile(join(first_target_dir, f))])
    first_hm_list = []

    for file_name in sorted(first_hm_files):
      file_dir = join(first_target_dir, file_name)
      f = open(file_dir, "rb")
      first_hm_list += [pickle.load(f)]
      f.close()


    #################### CALCULATE SCORES ####################
    first_target_scores = []
    second_target_scores = []


    # counter = 0
    for i in range(len(first_hm_list)):

        index = first_hm_files[i].split('_')[2]
        temp_data = data_list[patches_files.index(index)]
        t1, t2 = two_patch_corr_score(abs(first_hm_list[i]), temp_data)
               
        first_target_scores.append(t1)
        second_target_scores.append(t2)


    corr, pval = spearmanr(first_target_scores, second_target_scores)
    print(m, ': ', corr)


In [ ]:
from torch.nn.functional import softmax
from scipy.stats import spearmanr

print("Method: Repeated")

method_dir = '/media/heatmaps/repeated-new/'
methods = ['DeepLiftShap', 'ExtremalPertubation', 'FullGrad', 'GradCAM', 'Gradient',
          'GuidedBackProp', 'IntegratedGradients', 'GradCAMPlusPlus/layer4', 'IBA/layer4']

for m in methods:
    first_target_dir = join(method_dir, m, 'data')


    #################### READ HEATMAPS ####################
    first_hm_files = sorted([f for f in listdir(first_target_dir) if isfile(join(first_target_dir, f))])
    first_hm_list = []

    for file_name in sorted(first_hm_files):
      file_dir = join(first_target_dir, file_name)
      f = open(file_dir, "rb")
      first_hm_list += [pickle.load(f)]
      f.close()

    #################### CALCULATE SCORES ####################
    first_patch_scores = []
    second_patch_scores = []


    for i in range(len(first_hm_list)):

        index = first_hm_files[i].split('_')[2]
        temp_data = data_list[patches_files.index(index)]

        p1, p2 = two_patch_corr_score(abs(first_hm_list[i]), temp_data)
        first_patch_scores.append(p1)
        second_patch_scores.append(p2)

    corr, pval = spearmanr(first_patch_scores, second_patch_scores)
    print(m, corr)
